# "Candlestick" statistics & candlestick plot

***Candlestick*** is a type of chart commonly used in financial markets to represent the price movement of an asset, 
such as stocks or cryptocurrencies. It consists of individual "candles" that display the *opening*, 
*closing*, *high*, and *low* prices for a specific time period. 
Each candle has a rectangular body, representing the opening and closing prices, 
and thin lines, called wicks or shadows, indicating the highest and lowest prices during that time frame.

Candlestick charts often use colors to visually indicate whether the value within a specific candle has increased or decreased. For example, a positive price movement is typically represented by a green candle, indicating that the closing price is higher than the opening price (and by a red color otherwise). Basically "candlestick" main statistic is an indicator of whether the price has increased.

This guide describes comprehensively the process of building candlestick chart and all the details of its customization. 

This notebook uses definitions from [DataFrame](https://kotlin.github.io/dataframe/overview.html).

## Usage

Candlestick charts are widely used by traders and analysts to analyze price patterns and make informed decisions in the financial markets.

## Arguments

* Input (mandatory):
    - `x` — candle `x`-position (often time or date describing);
    - `open` — candle open value;
    - `high` — candle high value;
    - `low` — candle low value;
    - `close` — candle close value;

### Generalized signature

The specific signature depends on the function,
but all functions related to "candlestick" statistic
(which will be discussed further below - different variations of `statCandlestick()`, `candlestick()`)
have approximately the same signature with the arguments above:

```
statCandlestickArgs := 
   x, open, high, low, close
```

The possible types of `x`, `open`, `high`, `low`, `close` depend on where a certain function is used.
They can be simply `Iterable` (`List`, `Set`, etc.) or a reference to a column in a `DataFrame`
(`String`, `ColumnAccessor`) or the `DataColumn` itself.

## Output statistics

| name             | type    | description                                                              |
|------------------|---------|--------------------------------------------------------------------------|
| Stat.x           | X       | Candle `x`-position                                                      |
| Stat.open        | Double  | Candle open. Equals to input `open`.                                     |
| Stat.close       | Double  | Candle close. Equals to input `close`.                                   |
| Stat.min         | Double  | Candle minimum. Equals to input `low`.                                   |
| Stat.lower       | Double  | Candle lower value (i.e. smaller of `open`/`close`)                      |
| Stat.upper       | Double  | Candle lower value (i.e. greater of `open`/`close`)                      |
| Stat.max         | Double  | Candle maximum. Equals to input `high`.                                  |
| Stat.isIncreased | Boolean | Increase indicator: `true` if `close` value is greater than `open` value |

## StatCandlestick plots

In [1]:
%useLatestDescriptors
%use kandy
%use dataframe

In [2]:
// Create a simple candlestick dataset
val xList = listOf("Jan", "Feb", "Mar", "Apr", "May")
val openList = listOf(14.2, 6.7, 8.8, 11.2, 4.0)
val highList = listOf(15.5, 9.6, 10.7, 11.7, 9.9)
val lowList = listOf(7.5, 6.1, 8.5, 5.4, 4.0)
val closeList = listOf(8.0, 8.6, 10.7, 6.5, 9.8)
// Gather lists into df as columns
val df = dataFrameOf(
    "x" to xList,
    "open" to openList,
    "high" to highList,
    "low" to lowList,
    "close" to closeList,
)
df.head()

x,open,high,low,close
Jan,"14,200000","15,500000","7,500000","8,000000"
Feb,"6,700000","9,600000","6,100000","8,600000"
Mar,"8,800000","10,700000","8,500000","10,700000"
Apr,"11,200000","11,700000","5,400000","6,500000"
May,"4,000000","9,900000","4,000000","9,800000"


`df` has a signature

| x | open | high | low | close |
|---|------|------|-----|-------|

Let's take a look at `StatCandlestick` output DataFrame:

In [3]:
df.statCandlestick("x", "open", "high", "low", "close")

Stat,,,,,,,
x,open,close,min,lower,upper,max,isIncreased
Jan,"14,200000","8,000000","7,500000","8,000000","14,200000","15,500000",false
Feb,"6,700000","8,600000","6,100000","6,700000","8,600000","9,600000",true
Mar,"8,800000","10,700000","8,500000","8,800000","10,700000","10,700000",true
Apr,"11,200000","6,500000","5,400000","6,500000","11,200000","11,700000",false
May,"4,000000","9,800000","4,000000","4,000000","9,800000","9,900000",true


It has the following signature:

<table>
  <thead>
    <tr>
      <th alignt="left" colspan="8">Stat</th>
    </tr>
  </thead>
  <thead>
    <tr>
      <th>x</th>
      <th>open</th>
      <th>close</th>
      <th>min</th>
      <th>lower</th>
      <th>upper</th>
      <th>max</th>
      <th>isIncreased</th>
    </tr>
  </thead>
</table>

As you can see, we got a `DataFrame` with one `ColumnGroup` called `Stat` which contains several columns with statics.
For `statCandlestick`, each row corresponds to one candle.
`Stat.x` is the candle `x`-coordinate.
`Stat.open` and `Stat.close` correspond to candle open and close.
`Stat.min` and `Stat.max` correspond to candle low and high.
`Stat.lower` and `Stat.upper` correspond to candle edges.
`Stat.isIncreased` shows if value is increased (i.e `close > open`).

`DataFrame` with "candlestick" statistics is called `StatCandlestickFrame`

### `statCandlestick` context transform

`statCandlestick(statCandlestickArgs) { /*new plotting context*/ }` modifies a plotting context — instead of original data
(no matter was it empty or not) new `StatCandlestick` dataset (calculated on given arguments.
Inputs and weights can be provided as `Iterable` or as dataset column reference — by name as a `String`,
as a `ColumnReference` or as a `DataColumn`) is used inside a new context
(original dataset and primary context are not affected —
you can add layers using initial dataset outside the `statCandlestick` context).
Since the old dataset is irrelevant, we cannot use references for its columns.
But we can refer to the new ones.
They are all contained in the `Stat` group and can be called inside the new context:

In [4]:
plot {
    statCandlestick(xList, openList, lowList, highList, closeList) {
       errorBars {
            x(Stat.x)
            yMin(Stat.lower)
            yMax(Stat.upper)
            borderLine.color(Stat.isIncreased) {
                scale = categorical(true to Color.GREEN, false to Color.RED)
                legend.type = LegendType.None
            }
        }
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="o77nBI"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"color",
"values":["#3ba272","#ee6666"],
"limits":[true,false],
"guide":"none"
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"lower",
"ymax":"upper",
"color":"isIncreased"
},
"stat":"identity",
"data":{
"isIncreased":[false,true,true,false,true],
"upper":[14.2,8.6,10.7,11.2,9.8],
"lower":[8.0,6.7,8.8,6.5,4.0],
"x":["Jan","Feb","Mar","Apr","May"]
},
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"errorbar",
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"x"
},{
"type":"float",
"column":"lower"
},{
"type":"float",
"column":"upper"
},{
"type":"bool",
"column":"isIncreased"
}]
}
}],
"spec_id":"2"
};
 var containerDiv = document.getElementById("o77nBI");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Jan 
 
 
 
 
 
 
 
 
 Feb 
 
 
 
 
 
 
 
 
 Mar 
 
 
 
 
 
 
 
 
 Apr 
 
 
 
 
 
 
 
 
 May 
 
 
 
 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 8 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 12 
 
 
 
 
 
 
 14 
 
 
 
 
 
 
 
 
 y 
 
 
 
 
 x

### Candlestick layer

Basically, candlestick plot is a box plot where each box represents one candle. Box whisker's ends correspond to `low` and `high` values; and lower and upper edges to `open` and `close` (so here we need to determine which is greater and which is lesser — that's what we counted in the statistics). Non-positional attributes (most often color) indicate whether an increase or decrease has occurred.
So basically, we can build a candlestick with `statCandlestick` and `boxes` as follows:

In [5]:
import org.jetbrains.letsPlot.intern.toSpec

val statCandlestickBoxesPlot = df.plot {
    statCandlestick("x", "open", "high", "low", "close") {
        boxes {
            x(Stat.x)
            yMin(Stat.min)
            lower(Stat.lower)
            upper(Stat.upper)
            yMax(Stat.max)
            // temporary solution, middle shoudn't be necceasary
            middle(List(Stat.x.size()) { null })
            val colorScale = Scale.categorical(true to Color.GREEN, false to Color.RED)
            fillColor(Stat.isIncreased) {
                scale = colorScale
                // remove legend
                legend.type = LegendType.None 
            }
            borderLine.color(Stat.isIncreased) {
                scale = colorScale
                // remove legend
                legend.type = LegendType.None
            }
            alpha = 0.6
            // remove whisker ends
            whiskerWidth = 0.0
        }
    }
    layout.title = "`statCandlestick` + `boxes`"
}
statCandlestickBoxesPlot

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="9DIE6O"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"`statCandlestick` + `boxes`"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"fill",
"values":["#3ba272","#ee6666"],
"limits":[true,false],
"guide":"none"
},{
"aesthetic":"color",
"values":["#3ba272","#ee6666"],
"limits":[true,false],
"guide":"none"
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"upper":"upper",
"ymax":"max",
"middle":"middle",
"fill":"isIncreased",
"color":"isIncreased"
},
"stat":"identity",
"data":{
"min":[7.5,6.1,8.5,5.4,4.0],
"middle":[null,null,null,null,null],
"max":[15.5,9.6,10.7,11.7,9.9],
"isIncreased":[false,true,true,false,true],
"lower":[8.0,6.7,8.8,6.5,4.0],
"upper":[14.2,8.6,10.7,11.2,9.8],
"x":["Jan","Feb","Mar","Apr","May"]
},
"whisker_width":0.0,
"sampling":"none",
"alpha":0.6,
"inherit_aes":false,
"position":"dodge",
"geom":"boxplot",
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"x"
},{
"type":"float",
"column":"min"
},{
"type":"float",
"column":"lower"
},{
"type":"float",
"column":"upper"
},{
"type":"float",
"column":"max"
},{
"type":"bool",
"column":"isIncreased"
}]
}
}],
"spec_id":"5"
};
 var containerDiv = document.getElementById("9DIE6O");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Jan 
 
 
 
 
 
 
 
 
 Feb 
 
 
 
 
 
 
 
 
 Mar 
 
 
 
 
 
 
 
 
 Apr 
 
 
 
 
 
 
 
 
 May 
 
 
 
 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 8 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 12 
 
 
 
 
 
 
 14 
 
 
 
 
 
 
 16 
 
 
 
 
 
 
 
 
 `statCandlestick` + `boxes` 
 
 
 
 
 y 
 
 
 
 
 x

But we can do it even faster with `candlestick(statCandlestickArgs)` method:

In [6]:
val candlestickPlot = plot {
    candlestick(xList, openList, highList, lowList, closeList)
    layout.title = "`candlestick`"
}
candlestickPlot

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="iVuGto"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"`candlestick`"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"fill",
"values":["#3ba272","#ee6666"],
"limits":[true,false],
"guide":"none"
},{
"aesthetic":"color",
"values":["#3ba272","#ee6666"],
"limits":[true,false],
"guide":"none"
},{
"aesthetic":"linetype",
"values":[1.0,1.0],
"limits":[true,false],
"guide":"none"
},{
"aesthetic":"size",
"values":[0.6,0.6],
"limits":[true,false],
"guide":"none"
},{
"aesthetic":"alpha",
"values":[1.0,1.0],
"limits":[true,false],
"guide":"none"
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"fill":"isIncreased",
"color":"isIncreased",
"linetype":"isIncreased",
"size":"isIncreased",
"alpha":"isIncreased"
},
"stat":"identity",
"data":{
"min":[7.5,6.1,8.5,5.4,4.0],
"middle":[null,null,null,null,null],
"max":[15.5,9.6,10.7,11.7,9.9],
"isIncreased":[false,true,true,false,true],
"upper":[14.2,8.6,10.7,11.2,9.8],
"lower":[8.0,6.7,8.8,6.5,4.0],
"x":["Jan","Feb","Mar","Apr","May"],
"close":[8.0,8.6,10.7,6.5,9.8],
"open":[14.2,6.7,8.8,11.2,4.0]
},
"whisker_width":0.0,
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"boxplot",
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"x"
},{
"type":"float",
"column":"min"
},{
"type":"float",
"column":"lower"
},{
"type":"float",
"column":"upper"
},{
"type":"float",
"column":"max"
},{
"type":"float",
"column":"open"
},{
"type":"float",
"column":"close"
},{
"type":"bool",
"column":"isIncreased"
}]
},
"tooltips":{
"formats":[{
"field":"open",
"format":".2f"
},{
"field":"max",
"format":".2f"
},{
"field":"min",
"format":".2f"
},{
"field":"close",
"format":".2f"
}],
"title":"@x",
"lines":["open|@open","high|@max","low|@min","close|@close"],
"disable_splitting":true
}
}],
"spec_id":"8"
};
 var containerDiv = document.getElementById("iVuGto");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Jan 
 
 
 
 
 
 
 
 
 Feb 
 
 
 
 
 
 
 
 
 Mar 
 
 
 
 
 
 
 
 
 Apr 
 
 
 
 
 
 
 
 
 May 
 
 
 
 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 8 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 12 
 
 
 
 
 
 
 14 
 
 
 
 
 
 
 16 
 
 
 
 
 
 
 
 
 `candlestick` 
 
 
 
 
 
 
 
 
 
 x

Let's compare them:

In [7]:
plotGrid(listOf(statCandlestickBoxesPlot, candlestickPlot))

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="abZW19"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"layout":{
"name":"grid",
"ncol":2,
"nrow":1,
"fit":true,
"align":false
},
"figures":[{
"ggtitle":{
"text":"`statCandlestick` + `boxes`"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"fill",
"values":["#3ba272","#ee6666"],
"limits":[true,false],
"guide":"none"
},{
"aesthetic":"color",
"values":["#3ba272","#ee6666"],
"limits":[true,false],
"guide":"none"
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"upper":"upper",
"ymax":"max",
"middle":"middle",
"fill":"isIncreased",
"color":"isIncreased"
},
"stat":"identity",
"data":{
"min":[7.5,6.1,8.5,5.4,4.0],
"middle":[null,null,null,null,null],
"max":[15.5,9.6,10.7,11.7,9.9],
"isIncreased":[false,true,true,false,true],
"upper":[14.2,8.6,10.7,11.2,9.8],
"lower":[8.0,6.7,8.8,6.5,4.0],
"x":["Jan","Feb","Mar","Apr","May"]
},
"whisker_width":0.0,
"sampling":"none",
"alpha":0.6,
"inherit_aes":false,
"position":"dodge",
"geom":"boxplot",
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"x"
},{
"type":"float",
"column":"min"
},{
"type":"float",
"column":"lower"
},{
"type":"float",
"column":"upper"
},{
"type":"float",
"column":"max"
},{
"type":"bool",
"column":"isIncreased"
}]
}
}],
"spec_id":"12"
},{
"ggtitle":{
"text":"`candlestick`"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"fill",
"values":["#3ba272","#ee6666"],
"limits":[true,false],
"guide":"none"
},{
"aesthetic":"color",
"values":["#3ba272","#ee6666"],
"limits":[true,false],
"guide":"none"
},{
"aesthetic":"linetype",
"values":[1.0,1.0],
"limits":[true,false],
"guide":"none"
},{
"aesthetic":"size",
"values":[0.6,0.6],
"limits":[true,false],
"guide":"none"
},{
"aesthetic":"alpha",
"values":[1.0,1.0],
"limits":[true,false],
"guide":"none"
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"fill":"isIncreased",
"color":"isIncreased",
"linetype":"isIncreased",
"size":"isIncreased",
"alpha":"isIncreased"
},
"stat":"identity",
"data":{
"min":[7.5,6.1,8.5,5.4,4.0],
"middle":[null,null,null,null,null],
"max":[15.5,9.6,10.7,11.7,9.9],
"isIncreased":[false,true,true,false,true],
"upper":[14.2,8.6,10.7,11.2,9.8],
"lower":[8.0,6.7,8.8,6.5,4.0],
"x":["Jan","Feb","Mar","Apr","May"],
"close":[8.0,8.6,10.7,6.5,9.8],
"open":[14.2,6.7,8.8,11.2,4.0]
},
"whisker_width":0.0,
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"boxplot",
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"x"
},{
"type":"float",
"column":"min"
},{
"type":"float",
"column":"lower"
},{
"type":"float",
"column":"upper"
},{
"type":"float",
"column":"max"
},{
"type":"float",
"column":"open"
},{
"type":"float",
"column":"close"
},{
"type":"bool",
"column":"isIncreased"
}]
},
"tooltips":{
"formats":[{
"field":"open",
"format":".2f"
},{
"field":"max",
"format":".2f"
},{
"field":"min",
"format":".2f"
},{
"field":"close",
"format":".2f"
}],
"title":"@x",
"lines":["open|@open","high|@max","low|@min","close|@close"],
"disable_splitting":true
}
}],
"spec_id":"13"
}],
"kind":"subplots"
};
 var containerDiv = document.getElementById("abZW19");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(f

These two plots are almost identical (the only difference in tooltips).
Indeed, `candlestick` just uses `statCandlestick` and `boxes` and performs aesthetic mappings under the hood.

### `candlestick` customization

We can customize candlestick layer: `candlestick()` optionally opens a new context,
where we can configure it. We can set different color, borderline color, etc. for candles with increasing and decreasing value, or make general settings (as in the usual context opened by `boxes { ... }`):

In [8]:
df.plot {
    candlestick(x, open, high, low, close) {
        // Boxes context + StatCandlestick context
        // change fill color when increased
        increase {
            fillColor = Color.hex("#00FFFF")
        }
        // change fill color when decreased
        decrease.fillColor = Color.hex("#FF0000")
        // set constant border line color for all candles
        borderLine.color = Color.GREY
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="DW84xV"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"fill",
"values":["#00FFFF","#FF0000"],
"limits":[true,false],
"guide":"none"
},{
"aesthetic":"color",
"values":["#3ba272","#ee6666"],
"limits":[true,false],
"guide":"none"
},{
"aesthetic":"linetype",
"values":[1.0,1.0],
"limits":[true,false],
"guide":"none"
},{
"aesthetic":"size",
"values":[0.6,0.6],
"limits":[true,false],
"guide":"none"
},{
"aesthetic":"alpha",
"values":[1.0,1.0],
"limits":[true,false],
"guide":"none"
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"fill":"isIncreased",
"color":"isIncreased",
"linetype":"isIncreased",
"size":"isIncreased",
"alpha":"isIncreased"
},
"stat":"identity",
"data":{
"middle":[null,null,null,null,null],
"min":[7.5,6.1,8.5,5.4,4.0],
"max":[15.5,9.6,10.7,11.7,9.9],
"isIncreased":[false,true,true,false,true],
"upper":[14.2,8.6,10.7,11.2,9.8],
"lower":[8.0,6.7,8.8,6.5,4.0],
"x":["Jan","Feb","Mar","Apr","May"],
"close":[8.0,8.6,10.7,6.5,9.8],
"open":[14.2,6.7,8.8,11.2,4.0]
},
"whisker_width":0.0,
"color":"#a39999",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"boxplot",
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"x"
},{
"type":"float",
"column":"min"
},{
"type":"float",
"column":"lower"
},{
"type":"float",
"column":"upper"
},{
"type":"float",
"column":"max"
},{
"type":"float",
"column":"open"
},{
"type":"float",
"column":"close"
},{
"type":"bool",
"column":"isIncreased"
}]
},
"tooltips":{
"formats":[{
"field":"open",
"format":".2f"
},{
"field":"max",
"format":".2f"
},{
"field":"min",
"format":".2f"
},{
"field":"close",
"format":".2f"
}],
"title":"@x",
"lines":["open|@open","high|@max","low|@min","close|@close"],
"disable_splitting":true
}
}],
"spec_id":"17"
};
 var containerDiv = document.getElementById("DW84xV");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Jan 
 
 
 
 
 
 
 
 
 Feb 
 
 
 
 
 
 
 
 
 Mar 
 
 
 
 
 
 
 
 
 Apr 
 
 
 
 
 
 
 
 
 May 
 
 
 
 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 8 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 12 
 
 
 
 
 
 
 14 
 
 
 
 
 
 
 16 
 
 
 
 
 
 
 
 
 
 
 
 
 
 x

However, it can also be done as with other statistical layers (i.e., through mappings from statistics from `StatCandlestick` dataset):

In [9]:
df.plot {
    candlestick(x, open, high, low, close) {
        // Boxes context + StatCandlestick context
        alpha(Stat.isIncreased) {
            scale = categorical(true to 1.0, false to 0.1)
            legend {
                name = ""
                breaksLabeled(true to "increase", false to "decrease")
            }
        }
        fillColor = Color.GREY
        borderLine.color = Color.GREY
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="wCtzks"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"alpha",
"breaks":[true,false],
"values":[1.0,0.1],
"name":"",
"limits":[true,false],
"labels":["increase","decrease"]
},{
"aesthetic":"fill",
"values":["#3ba272","#ee6666"],
"limits":[true,false],
"guide":"none"
},{
"aesthetic":"color",
"values":["#3ba272","#ee6666"],
"limits":[true,false],
"guide":"none"
},{
"aesthetic":"linetype",
"values":[1.0,1.0],
"limits":[true,false],
"guide":"none"
},{
"aesthetic":"size",
"values":[0.6,0.6],
"limits":[true,false],
"guide":"none"
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"alpha":"isIncreased",
"fill":"isIncreased",
"color":"isIncreased",
"linetype":"isIncreased",
"size":"isIncreased"
},
"stat":"identity",
"data":{
"middle":[null,null,null,null,null],
"min":[7.5,6.1,8.5,5.4,4.0],
"max":[15.5,9.6,10.7,11.7,9.9],
"isIncreased":[false,true,true,false,true],
"lower":[8.0,6.7,8.8,6.5,4.0],
"upper":[14.2,8.6,10.7,11.2,9.8],
"x":["Jan","Feb","Mar","Apr","May"],
"close":[8.0,8.6,10.7,6.5,9.8],
"open":[14.2,6.7,8.8,11.2,4.0]
},
"whisker_width":0.0,
"color":"#a39999",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"boxplot",
"fill":"#a39999",
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"x"
},{
"type":"float",
"column":"min"
},{
"type":"float",
"column":"lower"
},{
"type":"float",
"column":"upper"
},{
"type":"float",
"column":"max"
},{
"type":"float",
"column":"open"
},{
"type":"float",
"column":"close"
},{
"type":"bool",
"column":"isIncreased"
}]
},
"tooltips":{
"formats":[{
"field":"open",
"format":".2f"
},{
"field":"max",
"format":".2f"
},{
"field":"min",
"format":".2f"
},{
"field":"close",
"format":".2f"
}],
"title":"@x",
"lines":["open|@open","high|@max","low|@min","close|@close"],
"disable_splitting":true
}
}],
"spec_id":"20"
};
 var containerDiv = document.getElementById("wCtzks");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Jan 
 
 
 
 
 
 
 
 
 Feb 
 
 
 
 
 
 
 
 
 Mar 
 
 
 
 
 
 
 
 
 Apr 
 
 
 
 
 
 
 
 
 May 
 
 
 
 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 8 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 12 
 
 
 
 
 
 
 14 
 
 
 
 
 
 
 16 
 
 
 
 
 
 
 
 
 
 
 
 
 
 x 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 increase 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 decrease

### `candlestick` plot

`candlestick(statCandlestickArgs)` and `DataFrame.candlestick(statCandlestickArgs)` are a family of functions for fast plotting a candlestick.

In [10]:
candlestick(xList, openList, highList, lowList, closeList)

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="fpLYaW"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"fill",
"values":["#3ba272","#ee6666"],
"limits":[true,false],
"guide":"none"
},{
"aesthetic":"color",
"values":["#3ba272","#ee6666"],
"limits":[true,false],
"guide":"none"
},{
"aesthetic":"linetype",
"values":[1.0,1.0],
"limits":[true,false],
"guide":"none"
},{
"aesthetic":"size",
"values":[0.6,0.6],
"limits":[true,false],
"guide":"none"
},{
"aesthetic":"alpha",
"values":[1.0,1.0],
"limits":[true,false],
"guide":"none"
},{
"aesthetic":"y",
"name":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"fill":"isIncreased",
"color":"isIncreased",
"linetype":"isIncreased",
"size":"isIncreased",
"alpha":"isIncreased"
},
"stat":"identity",
"data":{
"min":[7.5,6.1,8.5,5.4,4.0],
"middle":[null,null,null,null,null],
"max":[15.5,9.6,10.7,11.7,9.9],
"isIncreased":[false,true,true,false,true],
"upper":[14.2,8.6,10.7,11.2,9.8],
"lower":[8.0,6.7,8.8,6.5,4.0],
"x":["Jan","Feb","Mar","Apr","May"],
"close":[8.0,8.6,10.7,6.5,9.8],
"open":[14.2,6.7,8.8,11.2,4.0]
},
"whisker_width":0.0,
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"boxplot",
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"x"
},{
"type":"float",
"column":"min"
},{
"type":"float",
"column":"lower"
},{
"type":"float",
"column":"upper"
},{
"type":"float",
"column":"max"
},{
"type":"float",
"column":"open"
},{
"type":"float",
"column":"close"
},{
"type":"bool",
"column":"isIncreased"
}]
},
"tooltips":{
"formats":[{
"field":"open",
"format":".2f"
},{
"field":"max",
"format":".2f"
},{
"field":"min",
"format":".2f"
},{
"field":"close",
"format":".2f"
}],
"title":"@x",
"lines":["open|@open","high|@max","low|@min","close|@close"],
"disable_splitting":true
}
}],
"spec_id":"23"
};
 var containerDiv = document.getElementById("fpLYaW");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Jan 
 
 
 
 
 
 
 
 
 Feb 
 
 
 
 
 
 
 
 
 Mar 
 
 
 
 
 
 
 
 
 Apr 
 
 
 
 
 
 
 
 
 May 
 
 
 
 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 8 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 12 
 
 
 
 
 
 
 14 
 
 
 
 
 
 
 16 
 
 
 
 
 
 
 
 
 y 
 
 
 
 
 x

In [11]:
df.candlestick("x", "open", "high", "low", "close")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="fhNSLW"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"fill",
"values":["#3ba272","#ee6666"],
"limits":[true,false],
"guide":"none"
},{
"aesthetic":"color",
"values":["#3ba272","#ee6666"],
"limits":[true,false],
"guide":"none"
},{
"aesthetic":"linetype",
"values":[1.0,1.0],
"limits":[true,false],
"guide":"none"
},{
"aesthetic":"size",
"values":[0.6,0.6],
"limits":[true,false],
"guide":"none"
},{
"aesthetic":"alpha",
"values":[1.0,1.0],
"limits":[true,false],
"guide":"none"
},{
"aesthetic":"y",
"name":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"fill":"isIncreased",
"color":"isIncreased",
"linetype":"isIncreased",
"size":"isIncreased",
"alpha":"isIncreased"
},
"stat":"identity",
"data":{
"min":[7.5,6.1,8.5,5.4,4.0],
"middle":[null,null,null,null,null],
"max":[15.5,9.6,10.7,11.7,9.9],
"isIncreased":[false,true,true,false,true],
"upper":[14.2,8.6,10.7,11.2,9.8],
"lower":[8.0,6.7,8.8,6.5,4.0],
"x":["Jan","Feb","Mar","Apr","May"],
"close":[8.0,8.6,10.7,6.5,9.8],
"open":[14.2,6.7,8.8,11.2,4.0]
},
"whisker_width":0.0,
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"boxplot",
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"x"
},{
"type":"float",
"column":"min"
},{
"type":"float",
"column":"lower"
},{
"type":"float",
"column":"upper"
},{
"type":"float",
"column":"max"
},{
"type":"float",
"column":"open"
},{
"type":"float",
"column":"close"
},{
"type":"bool",
"column":"isIncreased"
}]
},
"tooltips":{
"formats":[{
"field":"open",
"format":".2f"
},{
"field":"max",
"format":".2f"
},{
"field":"min",
"format":".2f"
},{
"field":"close",
"format":".2f"
}],
"title":"@x",
"lines":["open|@open","high|@max","low|@min","close|@close"],
"disable_splitting":true
}
}],
"spec_id":"26"
};
 var containerDiv = document.getElementById("fhNSLW");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Jan 
 
 
 
 
 
 
 
 
 Feb 
 
 
 
 
 
 
 
 
 Mar 
 
 
 
 
 
 
 
 
 Apr 
 
 
 
 
 
 
 
 
 May 
 
 
 
 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 8 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 12 
 
 
 
 
 
 
 14 
 
 
 
 
 
 
 16 
 
 
 
 
 
 
 
 
 y 
 
 
 
 
 x

In case you want to provide inputs using column selection DSL, it's a bit different from the usual one —
you should assign them throw invocation eponymous functions:

In [12]:
df.candlestick {
    x(x)
    open(open)
    high(high)
    low(low)
    close(close)
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="LXqgCg"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"fill",
"values":["#3ba272","#ee6666"],
"limits":[true,false],
"guide":"none"
},{
"aesthetic":"color",
"values":["#3ba272","#ee6666"],
"limits":[true,false],
"guide":"none"
},{
"aesthetic":"linetype",
"values":[1.0,1.0],
"limits":[true,false],
"guide":"none"
},{
"aesthetic":"size",
"values":[0.6,0.6],
"limits":[true,false],
"guide":"none"
},{
"aesthetic":"alpha",
"values":[1.0,1.0],
"limits":[true,false],
"guide":"none"
},{
"aesthetic":"y",
"name":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"fill":"isIncreased",
"color":"isIncreased",
"linetype":"isIncreased",
"size":"isIncreased",
"alpha":"isIncreased"
},
"stat":"identity",
"data":{
"min":[7.5,6.1,8.5,5.4,4.0],
"middle":[null,null,null,null,null],
"max":[15.5,9.6,10.7,11.7,9.9],
"isIncreased":[false,true,true,false,true],
"lower":[8.0,6.7,8.8,6.5,4.0],
"upper":[14.2,8.6,10.7,11.2,9.8],
"x":["Jan","Feb","Mar","Apr","May"],
"close":[8.0,8.6,10.7,6.5,9.8],
"open":[14.2,6.7,8.8,11.2,4.0]
},
"whisker_width":0.0,
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"boxplot",
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"x"
},{
"type":"float",
"column":"min"
},{
"type":"float",
"column":"lower"
},{
"type":"float",
"column":"upper"
},{
"type":"float",
"column":"max"
},{
"type":"float",
"column":"open"
},{
"type":"float",
"column":"close"
},{
"type":"bool",
"column":"isIncreased"
}]
},
"tooltips":{
"formats":[{
"field":"open",
"format":".2f"
},{
"field":"max",
"format":".2f"
},{
"field":"min",
"format":".2f"
},{
"field":"close",
"format":".2f"
}],
"title":"@x",
"lines":["open|@open","high|@max","low|@min","close|@close"],
"disable_splitting":true
}
}],
"spec_id":"29"
};
 var containerDiv = document.getElementById("LXqgCg");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Jan 
 
 
 
 
 
 
 
 
 Feb 
 
 
 
 
 
 
 
 
 Mar 
 
 
 
 
 
 
 
 
 Apr 
 
 
 
 
 
 
 
 
 May 
 
 
 
 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 8 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 12 
 
 
 
 
 
 
 14 
 
 
 
 
 
 
 16 
 
 
 
 
 
 
 
 
 y 
 
 
 
 
 x

Candlestick plot can be configured with `.configure {}` extension —
it opens a context that combines bars, `StatCandlestick` and plot context.
That means you can configure boxes settings, mappings using `StatCandlestick` dataset and any plot adjustments:

In [13]:
df.candlestick("x", "open", "high", "low", "close").configure {
    // Boxes + StatCandlestick + PlotBuilder
    // Can't add a new layer
    y.axis.limits = 3.0..17.0
    increase {
        borderLine.color = Color.BLUE
    }
    decrease.borderLine.color = Color.YELLOW
    borderLine.width = 2.5
    fillColor = Color.GREY
    alpha = 0.6
    // Can configure general plot adjustments
    layout {
        title = "Configured candlestick plot"
        size = 800 to 400
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="o6ofH2"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Configured candlestick plot"
},
"mapping":{
},
"coord":{
"name":"cartesian",
"flip":false,
"ylim":[3.0,17.0]
},
"data":{
"min":[7.5,6.1,8.5,5.4,4.0],
"middle":[null,null,null,null,null],
"max":[15.5,9.6,10.7,11.7,9.9],
"isIncreased":[false,true,true,false,true],
"lower":[8.0,6.7,8.8,6.5,4.0],
"upper":[14.2,8.6,10.7,11.2,9.8],
"x":["Jan","Feb","Mar","Apr","May"],
"close":[8.0,8.6,10.7,6.5,9.8],
"open":[14.2,6.7,8.8,11.2,4.0]
},
"ggsize":{
"width":800.0,
"height":400.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"fill",
"values":["#3ba272","#ee6666"],
"limits":[true,false],
"guide":"none"
},{
"aesthetic":"color",
"values":["#5470c6","#fac858"],
"limits":[true,false],
"guide":"none"
},{
"aesthetic":"linetype",
"values":[1.0,1.0],
"limits":[true,false],
"guide":"none"
},{
"aesthetic":"size",
"values":[0.6,0.6],
"limits":[true,false],
"guide":"none"
},{
"aesthetic":"alpha",
"values":[1.0,1.0],
"limits":[true,false],
"guide":"none"
},{
"aesthetic":"x",
"name":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"fill":"isIncreased",
"color":"isIncreased",
"linetype":"isIncreased",
"size":"isIncreased",
"alpha":"isIncreased"
},
"stat":"identity",
"whisker_width":0.0,
"size":2.5,
"sampling":"none",
"alpha":0.6,
"inherit_aes":false,
"position":"identity",
"geom":"boxplot",
"fill":"#a39999",
"tooltips":{
"formats":[{
"field":"open",
"format":".2f"
},{
"field":"max",
"format":".2f"
},{
"field":"min",
"format":".2f"
},{
"field":"close",
"format":".2f"
}],
"title":"@x",
"lines":["open|@open","high|@max","low|@min","close|@close"],
"disable_splitting":true
},
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"x"
},{
"type":"float",
"column":"min"
},{
"type":"float",
"column":"lower"
},{
"type":"float",
"column":"upper"
},{
"type":"float",
"column":"max"
},{
"type":"float",
"column":"open"
},{
"type":"float",
"column":"close"
},{
"type":"bool",
"column":"isIncreased"
}]
},
"spec_id":"32"
};
 var containerDiv = document.getElementById("o6ofH2");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 800.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Jan 
 
 
 
 
 
 
 
 
 Feb 
 
 
 
 
 
 
 
 
 Mar 
 
 
 
 
 
 
 
 
 Apr 
 
 
 
 
 
 
 
 
 May 
 
 
 
 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 8 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 12 
 
 
 
 
 
 
 14 
 
 
 
 
 
 
 16 
 
 
 
 
 
 
 
 
 Configured candlestick plot 
 
 
 
 
 y 
 
 
 
 
 x